# Fitting Proteins

In [43]:
import os
import copy
import glob
import itertools
from chemper.chemper_utils import check_smirks_to_reference, get_typed_molecules, create_tuples_for_clusters
from making_proteins import everything_from_fastas, print_order_type_data, at_least_one_passed, ParameterSystem, by_biggest_size, by_biggest_smirks
import cmiles
import json
from parmed.modeller import ResidueTemplate
from chemper.smirksify import Reducer, print_smirks

In [2]:
from openeye import oechem, oedepict
import IPython
from IPython.display import display, Image

alpha_list = [255, 51, 153, 117]
final_colors = list()

for a in alpha_list:
    for c in oechem.OEGetContrastColors():
        temp_c = c.CreateCopy()
        temp_c.SetA(a)
        final_colors.append(temp_c)

def depictMatch(mol, match=None, supH = True, idx_atoms=list(), width=500, height=200, fn=None,
               color1=oechem.OELightBlue, color2=oechem.OELightSalmon, include_bonds=True):
    
    atom_bond_set = oechem.OEAtomBondSet()
    for atom in mol.GetAtoms():
        if atom.GetIdx() in idx_atoms:
            atom_bond_set.AddAtom(atom)
            if not include_bonds:
                continue
            for bond in atom.GetBonds():
                nbr_atom = bond.GetNbr(atom)
                if (nbr_atom.GetIdx() in idx_atoms) and nbr_atom.GetIdx() > atom.GetIdx():
                    atom_bond_set.AddBond(bond)
    
    dopt = oedepict.OEPrepareDepictionOptions()
    dopt.SetDepictOrientation( oedepict.OEDepictOrientation_Horizontal)
    dopt.SetSuppressHydrogens(supH)
    oedepict.OEPrepareDepiction(mol, dopt)
    
    opts = oedepict.OE2DMolDisplayOptions(width, height, oedepict.OEScale_AutoScale)
    opts.SetAtomColorStyle(oedepict.OEAtomColorStyle_WhiteMonochrome)
    disp = oedepict.OE2DMolDisplay(mol, opts)
    
    # Highlight indexed atoms
    hstyle = oedepict.OEHighlightStyle_Cogwheel
    hcolor = oechem.OEColor(color1)
    if not atom_bond_set.IsEmpty():
        oedepict.OEAddHighlighting(disp, hcolor, hstyle, atom_bond_set)
    
    if match is not None:
        # Highlight whole match
        hstyle = oedepict.OEHighlightStyle_BallAndStick
        hcolor = oechem.OEColor(color2)
        oedepict.OEAddHighlighting(disp, hcolor, hstyle, match)
    
    img = oedepict.OEImage(width, height)
    oedepict.OERenderMolecule(img,disp)
    
    if fn is not None:
        ext = oechem.OEGetFileExtension(fn)
        if oedepict.OEIsRegisteredImageFile(ext):
            ofs = oechem.oeofstream()
            if not ofs.open(fn):
                print("OE could not open file due to error %s" % fn)
                ofs.close()
            else:
                oedepict.OERenderMolecule(ofs, ext, disp)
                ofs.close()
        else:
            print("OE could not handle extension on %s" % fn)
    return img

# ==================================================

# Reparsing json system

In [4]:
def convert_json_and_oeb(json_file, mol_dir='./mol_files/'):
    with open(json_file, 'r') as inputf:
        d = json.load(inputf)

    mol_dir = os.path.abspath(mol_dir)
    mol_files = [os.path.join(mol_dir, m) for m in d['mol_files']]
    mols = list()
    for mol_file in mol_files:
        mol = oechem.OEMol()
        ifs = oechem.oemolistream(mol_file)
        while oechem.OEReadMolecule(ifs,mol):
            mols.append(oechem.OEMol(mol))

    return mols, d['smirks_lists'], d['clusters']

In [13]:
json_test = './all_aminos_together/allIn1_big_99sbildn_lj_1mols.json'
json_test2 = './all_aminos_together/allIn1_small_99sbildn_lj_1mols.json'
mols, dsmirks, dclusters = convert_json_and_oeb(json_test)
mols2, dsmirks2, dclusters2 = convert_json_and_oeb(json_test2)

# Final Dictionary 

The final results I want are:

initial SMIRKS, reduced SMIRKS (None if failed), four sorting, all fragments so the dictionary will have:

**Fragement**
* ordering 
    - 'initial smirks'
    - 'final smirks' 
    

In [49]:
final_dict = dict()

In [ ]:
file_keys = [
    ('big', ['big_smirks', 'biggest_size']),
    ('small', ['small_smirks', 'small_size'])
]
for fn_label, cluster_orders in file_keys:
    fns = glob.glob('./all_aminos_together/allIn1_%s_99sbildn_*_1mols.json' % fn_label)
    print('='*80)
    print(' '*20,fn_label)
    print('='*80)
    for f in fns:
        frag = f.split('_')[-2]
        if frag == 'torsion':
            prefix = f.split('_')[-3]
            frag = '%s_%s' % (prefix, frag)
        print('-'*80)
        print(' '*30,frag)
        print('-'*80)
        mols, dsmirks, dclusters = convert_json_and_oeb(f)
        
        if frag not in final_dict:
            final_dict[frag] = dict()
        
        for order in cluster_orders:
            if order in final_dict[frag]:
                if 'output' in final_dict[frag][order]:
                    print('Already in dictionary ', frag, order)
                    continue
                    
            final_dict[frag][order] = dict()
            d = dsmirks[order][frag]
            type_list = [(l, s) for l,s in d['type_list']]
            final_dict[frag][order]['initial'] = type_list
            
            print('ORIGINAL')
            print_smirks(type_list)
            
            if not d['checked']:
                final_dict[frag][order]['output'] = None
                continue
                
            red = Reducer(type_list, mols, verbose=False)
            final_dict[frag][order]['output'] = red.run(10000)
            print('REDUCED')
            print_smirks(final_dict[frag][order]['output'])

                     big
--------------------------------------------------------------------------------
                               angle
--------------------------------------------------------------------------------
Already in dictionary  angle big_smirks
Already in dictionary  angle biggest_size
--------------------------------------------------------------------------------
                               bond
--------------------------------------------------------------------------------
Already in dictionary  bond big_smirks
Already in dictionary  bond biggest_size
--------------------------------------------------------------------------------
                               charge
--------------------------------------------------------------------------------
Already in dictionary  charge big_smirks
ORIGINAL

 Label                | SMIRKS 
 zz_-0.41570	X        | [#7H1X3x0!r+0A:1](-;!@[#1H0X1x0!r+0A])(-;!@[#6H0X3x0!r+0A](-;!@[#6H1,#6H2;!r;+0;X4;x0;A](-;!@[#1H0X1,#6H1X4,#

In [ ]:
import pickle
pickle.dump(final_dict, open('./all_aminos_together/reduced_smirks_dict.p', 'wb'))

In [63]:
for frag, frag_dict in final_dict.items():
    print('='*80)
    print(' '*30, frag)
    print('='*80)
    for order, order_dict in frag_dict.items():
        print('-'*80)
        print(' '*20, order)
        print('-'*80)
        if 'output' in order_dict:
            if order_dict['output'] is not None:
                print('REDUCED')
                print_smirks(order_dict['output'])
            else:
                print("No Reduction")
            


                               angle
--------------------------------------------------------------------------------
                     big_smirks
--------------------------------------------------------------------------------
REDUCED

 Label                | SMIRKS 
 zz_50.000	109.500    | [*:1]~[*:2]~[*:3] 
--------------------------------------------------------------------------------
 zz_70.000	120.000    | [*:1](~[*:2]~[#6x2,#7:3])~[*] 
--------------------------------------------------------------------------------
 zz_50.000	120.000    | [*:1]~[#6x2H1,#7H1,#7H2,#7x2H1:2](~[#1:3])~[#1x0H0,#6X4x0H1,#6H2x0,#6X3H0x3,#6x2H0,#6X3x2,#6x2H1,#7X2x2H0,#7x2H1] 
--------------------------------------------------------------------------------
 zz_40.000	109.500    | [#6:1]~[#6:2](~[#6:3])~;!@[*] 
--------------------------------------------------------------------------------
 zz_80.000	109.700    | [*;X3:1](~[#6:2]~[*:3]-[*])~;!@[*]~[*]~[#7] 
-------------------------------------------

In [65]:
for frag, frag_dict in final_dict.items():
    print('='*80)
    print(' '*30, frag)
    print('='*80)
    for order, order_dict in frag_dict.items():
        print('-'*80)
        print(' '*20, order)
        print('-'*80)
        if 'output' in order_dict:
            if order_dict['output'] is not None:
                print('REDUCED')
                print_smirks(order_dict['output'])
            else:
                print("No Reduction")

                               angle
--------------------------------------------------------------------------------
                     big_smirks
--------------------------------------------------------------------------------
REDUCED

 Label                | SMIRKS 
 zz_50.000	109.500    | [*:1]~[*:2]~[*:3] 
--------------------------------------------------------------------------------
 zz_70.000	120.000    | [*:1](~[*:2]~[#6x2,#7:3])~[*] 
--------------------------------------------------------------------------------
 zz_50.000	120.000    | [*:1]~[#6x2H1,#7H1,#7H2,#7x2H1:2](~[#1:3])~[#1x0H0,#6X4x0H1,#6H2x0,#6X3H0x3,#6x2H0,#6X3x2,#6x2H1,#7X2x2H0,#7x2H1] 
--------------------------------------------------------------------------------
 zz_40.000	109.500    | [#6:1]~[#6:2](~[#6:3])~;!@[*] 
--------------------------------------------------------------------------------
 zz_80.000	109.700    | [*;X3:1](~[#6:2]~[*:3]-[*])~;!@[*]~[*]~[#7] 
-------------------------------------------